# Example Notebook for Network Visualization

### Change working directory to this root

In [1]:
import os
os.chdir("..")

### Imports

In [ ]:
import random
from typing import Any

import torch
from torch.utils.data import DataLoader, _utils
import matplotlib.pyplot as plt
import torchaudio
import IPython

from utils.config import load_cfg_from_hydra
from utils.containers import MelSpecParameters
from models.mel_spec_converters import SimpleMelSpecConverter

### Load model

In [ ]:
# weights path
import hydra

from models.modules.base import load_inner_model_state_dict

weights_path = "weights/lvl1_vqvae.ckpt"

# Infer device
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Current device is {device}")

# Load network
cfg = load_cfg_from_hydra(config_path="../config", config_name="lvl1_vqvae_small")
cfg.learning.batch_size = 128
cfg.learning.val_split = 1.0 # Will it let me do that?
model = hydra.utils.instantiate(cfg.module, _convert_="partial").to(device)
model = load_inner_model_state_dict(model, weights_path)

### Load data loader

In [ ]:
dataset = hydra.utils.instantiate(cfg.data.dataset, _recursive_=False)
dataset_len = len(dataset)
slice_len = 128

# Select a random sample
sample_start = random.randint(0, dataset_len - slice_len - 1)
dataset_slice = _utils.collate.default_collate([dataset[i] for i in range(sample_start, sample_start + slice_len)])
concatenated_slice = dataset_slice["slice"] # type: ignore

print(f"The concatenated slice dimensions are {concatenated_slice.shape}")

### WAV Player element

In [5]:
# this is a wrapper that take a filename and publish an html <audio> tag to listen to it

def wavPlayer(filepath):
    """ will display html 5 player for compatible browser

    Parameters :
    ------------
    filepath : relative filepath with respect to the notebook directory ( where the .ipynb are not cwd)
                of the file to play

    The browser need to know how to play wav through html5.

    there is no autoplay to prevent file playing when the browser opens
    """
    
    src = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>Simple Test</title>
    </head>
    
    <body>
    <audio controls="controls" style="width:600px" >
        <source src="files/%s" type="audio/mp3" />
        Your browser does not support the audio element.
    </audio>
    </body>
    """%(filepath)
    display(HTML(src))

### Initialize Mel-Spectrogram Converter

In [6]:
n_mels = 32
mel_spec_params = MelSpecParameters(n_fft=256, f_min=0, hop_length=64, n_mels=n_mels, power=1.0, pad=0)
mel_spec_converter = SimpleMelSpecConverter(mel_spec_params)

lin_vector = torch.linspace(
    1.0,
    20.0,
    n_mels,
)
eye_mat = torch.diag(lin_vector).to(device)

### Load data point and visualize it

In [ ]:
image_reshaped = concatenated_slice.flatten().cpu().numpy() # type: ignore

plt.figure(figsize=(30, 5))
plt.plot(image_reshaped)

plt.tight_layout()
plt.ylim(-1.2, 1.2)
plt.show()

plt.figure(figsize=(30, 5))
plt.matshow(torch.tanh(eye_mat @ mel_spec_converter.convert(torch.tensor(image_reshaped))).cpu().numpy(), 
            origin='lower', aspect='auto', vmin=0, vmax=1)
plt.show()

torchaudio.save('sample.mp3', concatenated_slice.flatten().unsqueeze(0).cpu().detach(), 44100, format='mp3') # type: ignore
IPython.display.Audio(filename="sample.mp3") # type: ignore

## Inference

### Perform inference

In [ ]:
collated_batch = {"slice": concatenated_slice.to(device)}
print(collated_batch["slice"].shape)
with torch.no_grad():
    reconstructed_slice = model(collated_batch)
reconstructed_slice_flattened = reconstructed_slice["slice"].flatten().cpu().detach()
print(torch.nn.functional.smooth_l1_loss(reconstructed_slice["slice"], collated_batch["slice"]))

plt.figure(figsize=(30, 5))
plt.plot(reconstructed_slice_flattened)

plt.tight_layout()
plt.ylim(-1.2, 1.2)
plt.show()

plt.figure(figsize=(30, 5))
plt.matshow(torch.tanh(eye_mat @ mel_spec_converter.convert(torch.tensor(reconstructed_slice["slice"].flatten()))).cpu().numpy(), 
            origin='lower', aspect='auto', vmin=0, vmax=1)
plt.show()

torchaudio.save('sample.mp3', reconstructed_slice["slice"].flatten().unsqueeze(0).cpu().detach(), 44100, format='mp3') # type: ignore
IPython.display.Audio(filename="sample.mp3") # type: ignore

### Compare mel specs of a single 1024 length slice

In [ ]:
single_sample = dataset[sample_start]
collated_sample = {key: torch.stack((value,), dim=0).to(device) for (key, value) in single_sample.items() if key == "slice"}

with torch.no_grad():
    reconstructed_single_sample = model(collated_sample)
    
plt.figure(figsize=(30, 5))
plt.plot(collated_sample["slice"].flatten().cpu().detach().numpy())

plt.tight_layout()
plt.ylim(-1.2, 1.2)
plt.show()

plt.figure(figsize=(30, 5))
plt.plot(reconstructed_single_sample["slice"].flatten().cpu().detach().numpy())

plt.tight_layout()
plt.ylim(-1.2, 1.2)
plt.show()
    
plt.figure(figsize=(30, 5))
plt.matshow(
    torch.tanh(
        eye_mat @ mel_spec_converter.convert(collated_sample["slice"].flatten())
    ).cpu().numpy(), origin='lower', aspect='auto', vmin=0, vmax=1
)
plt.show()

plt.figure(figsize=(30, 5))
plt.matshow(
    torch.tanh(
        eye_mat @ mel_spec_converter.convert(torch.tensor(reconstructed_single_sample["slice"].flatten()))
    ).cpu().numpy(), origin='lower', aspect='auto', vmin=0, vmax=1
)
plt.show()